#Gazebo편#

[1편 기초편](https://colab.research.google.com/drive/10-LOBTYDF35S9GHNhl9FXcRC4Uwpi9mL)

[2편 초급응용과 MAVROS기초편](https://colab.research.google.com/drive/1Ej2BcMExu4QmwEPh1r1y_cu2Q5fGqEF0)

3편은 사진이 필요한 부분이 많아 상당부분 링크로 대체

#Gazebo 기초 모델제작 튜토리얼

http://gazebosim.org/tutorials?cat=guided_b&tut=guided_b3

# Gazebo <-> ROS 센서 (kinect) 생성 튜토리얼

http://gazebosim.org/tutorials?tut=ros_depth_camera&cat=connect_ros

### TroubleShooting ###

> .gazebo 폴더는 home 디렉토리에 숨김파일로 존재. 보이게 하려면 Ctrl+h

> kinect 폴더를 열게되면 model.sdf 말고도 다른 버전 파일들이 있는데
<br>신경쓰지 말고 model.sdf만 수정해 주면 됨

> 여기서 RVIZ로 테스트할 때 frame 목록이 자동 생성되지 않는것 처럼 보임
</br>따라서 frame에 직접 자신이 입력한 frame명을 입력해 주어야 함 (eg. camera_link)

> pointCloudCutoff 부분과 camera의 clip 부분을 수정해 주어야 함
</br>안그러면 너무 좁은 영역밖에 안보임

> 1편과 2편을 모두 돌려봤다면 별도 설치없이 실행 가능함

## 다른 모델과 통합##

은 굉장히 간단함. model editor에 들어가서 모델 두개 이상을 불러온 후

기초 모델제작 튜토리얼에서 했던 것 처럼 Joint 설정을 해주어 묶어서 저장해주면됨.

> 주의할점은, sensor나 로봇의 parameter를 설정할때, Original의 값을 변경한다고
</br>합친 모델에 적용되지 않음. 합친 모델의 model.sdf 에 들어가서 각 모델의 parameter를 바꾸어 주어야 함.

# Gazebo와 MAVROS 기본 연동 #

## 간단한 MAVROS와 GAZEBO 사이 Interaction ##

MAVROS와 GAZEBO에서 통신 topic과 자료형 그리고 link parameter만 알면 쉽게 만들 수 있다.

> 실행방법

  항상 시작할땐

    $ roscore
    
  그리고 2편에 MAVROS 노드 여는법을 통해서 pixhawk 연결하고 mavros 노드 열어준다.
    
    $ roslaunch gazebo_ros empty_world.launch
    
  그리고 gazebo 에서 가장 기본적인 cube 하나 만든다.

  아래 코드를 복사하여 ros package 의 src 폴더에 넣어준다. 보기 편하게 MAVROS가 설치된 package에 넣었다.
    
    파일명은 rotating_cube.py
    
  이 파일이 보이는 폴더에 들어가서 터미널을 열고
    
  실행 권한 부여
    
    $ chmod u+x rotating_cube.py

  Home Directory로 돌아가서
    
    $ rosrun 내package rotating_cube.py

In [ ]:
#!/usr/bin/env python
#-*- coding: utf-8 -*-
# ╔╦╗┌─┐┌─┐┬┌─┐┌┐┌┌─┐┌┬┐    
#  ║║├┤  └─┐││  ┬│││├┤    ││    
# ═╩╝└─┘└─┘┴└─┘┘└┘└─┘─┴┘    
# ┌┐  ┬  ┬  ╔╦╗┌─┐┌─┐┌─┐┌─┐┌┐┌
# ├┴┐└┬┘   ║║├─┤└─┐├┤  │  ││││
# └─┘  ┴    ═╩╝┴  ┴└─┘└─┘└─┘┘└┘
#Designed_by_Daseon_#

#rotating_cube.py

import rospy                                                                        # 로스 파이썬
from sensor_msgs.msg import Imu                                                     # Imu 형 메시지
from gazebo_msgs.msg import LinkState                                               # Gazebo LinkState 형 메시지

att_x = 0.                                                                          # 자세 초기화
att_y = 0.
att_z = 0.
att_w = 0.

def processing_fnc(msg):                                                            # Subscribe 콜백

    global att_x                                                                    # 변수 globalize
    global att_y
    global att_z
    global att_w

    att_x = msg.orientation.x                                                       # 받아온 메시지 각 축에 넣기
    att_y = msg.orientation.y                                                       # gazebo와 mavros의 자세 데이터가 모두 std geomeyry_msg
    att_z = msg.orientation.z                                                       # 이기 때문에 이렇게 그냥 넣어줄 수 있다. 이 메시지는
    att_w = msg.orientation.w                                                       # 자세를 쿼터니언 형태 로 표시한다. rostopic echo 혹은 info로 어떤 형태 데이터인지 알 수 있음

rospy.init_node('show_window')                                                      # ROS node 초기화

cube_att_dat = LinkState()                                                          # 자료형 대입
cube_att_dat.link_name = 'cube::link_0'                                             # ****link name 확인****

cube_att_dat.pose.position.x = 0.0                                                  # 기본 위치 catesian
cube_att_dat.pose.position.y = 0.0
cube_att_dat.pose.position.z = 5.0                                                  # 땅바닥에 질질 끌리지 않게 5m 띄움

cmd_cube_pub = rospy.Publisher('/gazebo/set_link_state', LinkState, queue_size=1)   # publisher 생성 후 advertise. gazebo에서 subscribe 하게된다.

aquiz_px4_att_dat = rospy.Subscriber('/mavros/imu/data', Imu, processing_fnc)       # subscriber 생성 MAVROS 에서 imu 데이터 받아온다.

rate = rospy.Rate(30)                                                               # 초당 30번 publish(while문)

while ~rospy.is_shutdown():                                                         # rospy가 실행중일때
    

    cube_att_dat.pose.orientation.x = att_x                                         # 차례차례 각 메시지에 태워준다.
    cube_att_dat.pose.orientation.y = att_y
    cube_att_dat.pose.orientation.z = att_z
    cube_att_dat.pose.orientation.w = att_w
    
    cmd_cube_pub.publish(cube_att_dat)                                              # 태운 메시지 publish 하기

    rate.sleep()                                                                    # 초당 30번 while문 돌아가게끔 쉬어준다.

rospy.spin()                                                                        # 제어 권한 되돌려주기. 없어도 된다.

## Software-in-the-loop simulation (MAVROS) ##

##Readme##

> 이 부분을 시작하기 위해서는 다음 페이지에서 개발자 tool chain을 설치해 주어야 한다. 자세한 방법 생략

[툴체인 설치](https://dev.px4.io/v1.9.0/en/setup/dev_env_linux.html)


이 링크가 공식 sitl simulation 도움말 페이지다
[developers' link](https://dev.px4.io/v1.9.0/en/simulation/ros_interface.html)

### 네트워크 설정 ###

    $ ifconfig
    
    를 통해서 이 컴퓨터의 ip 주소를 찾아낸다.
    
    그리고 아래 xxx.xxx.xxx.xxx 부분에 집어넣어 실행
    
    $ roslaunch mavros px4.launch fcu_url:="udp://:14540@xxx.xxx.xxx.xxx:14557"
    
  > 네트워크 그런거 신경쓰는 거 아니면
    </br> 회귀 IP인 127.0.0.1 로 쓰면 된다.
</br>(eg. $ roslaunch mavros px4.launch fcu_url:="udp://:14540@127.0.0.1:14557"
    

    
    



###실행###

로스코어를 실행한 후 Firmware 경로로 이동한다. 이 이후의 모든 명령은 Firmware 경로에서 실행된다.

gazebo 모델 빌드 하지만 화면은 나타나지 않는다.
    
    $ no_sim=1 make px4_sitl_default gazebo
    
    이때 열린 창을 A 창이라고 하자.
    
새 터미널을 열어 gazebo 환경변수 셋업

    $ source Tools/setup_gazebo.bash $(pwd) $(pwd)/build/px4_sitl_default
    
시뮬레이션 기본 월드 열기

    $ roslaunch gazebo_ros empty_world.launch world_name:=$(pwd)/Tools/sitl_gazebo/worlds/iris.world
    
아까 열렸던 A 창에

    pxh> commander takeoff
    
    입력.
    
이륙을 하는동안 새 창에서 다음을 입력하여 ROS에 데이터가 제대로 들어가는지 확인해 본다.

    $ rostopic list
    
    $ rostopic echo /mavros/global_position/local
    
이륙이 잘 되는지 확인한다. QGC를 켜지 않았다면 이륙 후 Failsafe가 작동되어

저절로 착륙 되는것이 맞다.

### Trouble shooting ###
>가끔 문제가 생겨서 gazebo를 강제종료 할 일이 생기는데 그리고 다시 gazebo를 켜면
</br>까만화면만 뜨는 경우가 있다.</br>이때 지금까지 실행시킨거랑 roscore랑 다 꺼주고 다시 시작하자.
    
    



그런데 우리가 사실 드론 나는게 궁금한게 아니었다.

우리는 드론이 나는와중의 외부 센서 데이터들을 받아오고 싶다. 예를들어 비행 모델에 Kinect 같은것을 달아야 한다.

# MAVROS로 GAZEBO상에서 SITL이 가능하도록 </br>임의의 센서를 장착한 모델의 simulation world 제작하기.#


PIXHAWK에서 SITL을 하기 위한 많은 모델들이 존재한다. 

하지만 드론이 나는것과 별개로 별도의 임무 수행을 하는 장비를 부착한 채로 시뮬레이션이 하고 싶다면?

Gazebo에서 model에디터로 두 모델을 합쳐서 새로운 model을 만들고 그 model을 기반으로 새로운 World를 만들어 시뮬레이션을 돌리면 된다고 생각이 될수도 있으나, 해봤더니 안된다. 

두개의 모델을 합쳐서 하나의 모델을 만드는 것이 아니라. 

독립된 두개의 모델을 world 파일 차원에서 수정을 통해서 묶어 주어야한다.

> 이건 어디서도 못배운다(= 찾아보긴했는데 못봤음). 
</br>다선이의 시행착오와 연구끝에 태어난 방법이다.


우선 기본적으로 ~/Firmware/Tools/sitl_gazebo/worlds 폴더에 기본 제공되는 SITL 모델들이 있다.

우리의 목적은 시뮬레이션 모델 iris와 depth camera kinect를 합쳐서 

SITL을 통한 비행을 하면서 RGBD 데이터를 받아 오는 model이 들어간 world를 만드는 것으로 한다.



다음과 같이 따라하자. 자세한 설명은 생략하고 필요하면 다선이에게 물어보자

###1.
위에서 만들었던 ros 연동 kinect 모델 폴더를 kinect_test 라는 이름으로
</br> ~/Firmware/Tools/sitl_gazebo/models 폴더에 넣어준다.

>내부 파일명도 기존에서 kinect_test로 바꿔주고 파일 내부에 선언되있는 이름들도 kinect_test로 바꿔주자.





###2.
~/Firmware/Tools/sitl_gazebo/worlds 폴더로 들어간다.

쉽게쉽게 하기 위해 그나마 비슷해 보이는 iris_fpv_cam.world 파일을 복제하여 iris_kinect_test.world 파일을 만들자.

파일을 열어 uri include 부분의 iris_fpv_cam를 iris_kinect_test로 바꾸자.



###3.
앞서 언급했던 models 폴더에서 또한 iris_fpv_cam 폴더를 복사해
iris_kinect_test로 이름을 바꾸자. 

iris_kinect_test 폴더를 들어가 iris_fpv_cam.sdf 이름을 iris kinect_test.sdf로 바꾸고 파일을 연다.

그리고 다음 xml 스크립트로 덮어쓰자.

In [ ]:
<?xml version='1.0'?>
<sdf version='1.5'>
  <model name='iris_kinect_test'>

    <include>
      <uri>model://iris</uri>
    </include>

    <include>
      <uri>model://kinect_test</uri>
      <pose>0.05 0 0.1 0 0 0</pose>
    </include>
    <joint name="kinect_test_joint" type="fixed">
      <child>kinect_test::link</child>
      <parent>iris::base_link</parent>
      <axis>
        <xyz>0 0 1</xyz>
        <limit>
          <upper>0</upper>
          <lower>0</lower>
        </limit>
      </axis>
    </joint>

  </model>
</sdf>

iris와 kinect_test를 불러오며 kinect_test 모델과 joint가 생성될 위치와 오일러앵글은 아래 pose에 선언된다.

그리고 joint 부분에서 fixed 형태로 체결해준다.

###TroubleShooting
>kinect 모델 파일들 내부에서 이름을 제대로 바꾸지 않았다면</br> child 의 link name이 달라질 수도 있다. gazebo에서 확인하거나 파일내부 name을 바꾸자

###4.
해당 폴더의 model.config 파일에서 역시

iris_fpv_cam.sdf를 iris_kinect_test.sdf로 바꿔주자.

###5.

worlds 폴더에 iris_kinect_test.world가 있어야 하고

models 폴더에 kinetic_test 폴더가 있어야하고

models 폴더에 iris_kinect_test 폴더가 있어야 하는 동시에 내부 파일들이 수정되어 있어야 한다.

###6.
테스트하자

터미널 열고

    $ roscore
    
새 터미널 열고

    $ roslaunch mavros px4.launch fcu_url:="udp://:14540@127.0.0.1:14557"
    
새 터미널(터미널A로 부른다) 열고

    $ cd Firmware
    
    $ no_sim=1 make px4_sitl_default gazebo

새 터미널 열고

    $ roslaunch gazebo_ros empty_world.launch world_name:=$(pwd)/Tools/sitl_gazebo/worlds/iris_kinect_test.world
    
울퉁불퉁한 땅과 iris 위에 kinect가 불안하게 부착된 모델이 나와야 한다.

새 터미널 열고

    $ rosrun rviz rviz
    
fixed frame 을 camera_link로 바꿔준다. (직접 쳐야한다)

point cloud나 depth cloud를 만들어보자.

그리고 아까 A 터미널에서 다음과 같이 입력한다.

    pxh> commander takeoff
    
관성모멘트와 질량이 변해 개판으로 나는걸 볼 수 있다.

한편 rviz에는 kinect의 data가 들어오고 있음을 볼 수 있다.

###TroubleShooting
>simulation을 종료하고 다시시작하고 싶을때 다른방법을 알아보고 있는중이지만</br> ros 관련된 모든 노드를 닫은 후 재시작 해야하는것 같다.</br> 안그러면 에러 나거나 까만 화면만 뜬다.
    

#GAZEBO - PIXHAWK HILS#

Gazebo Hils는 Pixhawk의 자체적인 sensor 적용을 Disable 시키고 Gazebo에서 센서값을 수신, 픽스호크에서 처리를 통해

Gazebo로 커맨드를 전송하여 구현된다. 이 과정에서 14550포트의 UDP를 이용하여 픽스호크와 QGC와 연결이 가능하다.

## Let us begin ##

가장 처음에 해야 할 것은 픽스호크에서 QGC를 이용해, 설정을 바꿔주는 것이다.

>Daily Build 로 설정했기 때문에, 메뉴 위치가 상이할 수 있다.

1. USB를 통해 컴퓨터와 연결해 준다. ttyACM 으로 나타나게끔, 마이크로 5핀으로 해준다.

2. 톱니바퀴 모양 탭으로 들어가서 Safety 메뉴에서 HIL 이나 HILT을 찾아 Enable 해준다.

3. 같은 탭에서 Airframe 메뉴로 들어가 simulation(Copter) 에서 HIL Quadcopter 옵션을 선택하고 활성화 시킨다.

4. 맨 왼쪽 탭을 눌러 General 메뉴에서 AutoConnect 설정을 UDP를 활성화 시키고 나머지는 전부 비활성화 한다.

그 다음엔 컴퓨터 쪽에서 설정해 준다. 

1. Firmware폴더를 복사하여 다른이름으로 바꿔준다. (eg. Firmware_HIL)

2. 그리고 그 폴더를 다음 명령어를 통하여 빌드해 준다.


    $ DONT_RUN=1 make px4_sitl_default gazebo
    
이제 파일을 수정해야 한다. Firmware_HIL(예시) 폴더의 Gazebo 모델이 SITL에 맞게 되어있기 때문에, 몇가지 바꿔서 HITL이 가능하도록 해준다.

1. Firmware_HIL 폴더에서 ( Tools/sitl_gazebo/models/iris/iris.sdf ) 경로의 파일을 gedit이나 다른 수정 프로그램으로 열어준다.

2. ctrl + f 하여 serial 을 검색하고 다음과같이 항목을 수정한다. 
  
    1) serialEnabled 항목을 true 로 바꿔준다.
    
    2) serialDevice 항목을 
    
        $ls /dev/ttyACM* 명령어에서 나타나는 명칭으로 바꿔준다.
        
        (eg. /dev/ttyACM0)
    3) hil_mode 항목을 true로 바꿔준다.

이제 준비는 끝났다. 새로운 터미널을 열어 다음과 같이 해주자.

환경변수 셋업

    $ source Tools/setup_gazebo.bash $(pwd) $(pwd)/build/px4_sitl_default
    
가지보 HILS 실행
    
    $ gazebo Tools/sitl_gazebo/worlds/iris.world
    
가지보가 열렸으면 QGC를 실행 시키고 자동으로 연결 되는지 확인하고 안된다면 수동으로 연결해 보고 그래도 안된다면 다선이에게로


##MAVROS와 함께##

환경변수 셋업까지 마치고 gazebo-ros로 들어가준다.

    $ roslaunch gazebo_ros empty_world.launch
    
환경변수 셋업을 완료 했다면 import에 3DR iris가 보일것이다.

가져다가 넣으면 된다.

카메라가 달린 모델이나 센서가 달린 모듈 모두 우리가 수정한 iris에 기반하고 있기 때문에 사용 가능하고

이전에 만들었었던 kinect 달았던 iris도 사용 가능하다.

그러면 mavros를 회귀 아이피를 이용해 연결해 준다.

    $ roslaunch mavros px4.launch fcu_url:="udp://:14540@127.0.0.1:14550"
    
이게 어떤걸 먼저 끄고 실행 시키냐에 영향을 많이 받지만 그 순서를 확실히는 잘 모르겠다.

만약 다른 월드를 만들어야 겠다면 다시 다 끄고 다시시작하는게 가장 좋은방법으로 사료된다.

우리가 만들었던 iris with kinect를 가져와서

    $ rostopic list
    
로 토픽 점검도 해보고

    $ rosrun rviz rviz
    
를 통해 rviz를 실행시켜 카메라가 별도로 잘 작동되는지 확인해 본다.

## Landing 학습데이터 수집 환경 제작 프로젝트(작성중)

In [ ]:
#!/usr/bin/env python
#-*- coding: utf-8 -*-
                                                       
################################################################################################################################################################################ -ohh`  #####################                
#                                                                                                                                                                             -ohNMMMMd`                    #
#                                                                                                                                                                         -ohNMMMMMmMMMd`                   #
#                                                                                                                                                                     -ohNMMMMNho-  yMMMd`                  #
#                                                                                                                                                                 -ohNMMMMNho-       yMMMd`                 #
#                                                                                                                                                             -ohNMMMMNho-            yMMMd`                #
#                                                                                                                                                         -+hNMMMMMdo-                 yMMMd`               #
#                                                                                                                                                     -+hNMMMMMdo:`                     yMMMd`              #
#                                                                                                                                                 -+hNMMMMMho-                           sMMMd`             #
#                                                                                                                                                `dMMMMho-                                sMMMd`            #
#                                                                                                                                                 `dMMMo                                   sMMMd`           #
#                                                                                                                                                  `dMMMs                                   sMMMd`          #
#                                                                                                                                                   `dMMMs                                   sMMMm`         #
#                                                                                                                                                    `dMMMo         `:osyyo/.                 sMMMm`        #
#                                                                                                                                                     `dMMMo      `sNMMMMMMMMh-                sMMMm`       #
#    MMMMMMMMo mMMMMNNds-    -yNMMMMNd-      sMMs                -MMh                                    `ddh                                          `dMMMo    `mMMMMMMMMMMMM/                sMMMm`      #
#    MMM/----. mMMo--+mMMy  oMMN+.`./h:      sMMy      -+osso/.  -MMh-+so:    :osso/`  .++:`/+`-+osso/. `+MMN++/  -+sso/.  `++/`:+/++-   :++.           `dMMMo   oMMMMMMMMMMMMMm                 sMMMm.     #
#    MMMyssss` mMM/   `mMM/.MMM-             sMMy      oysosmMM/ -MMNhydMMy -mMNsodMN/ /MMNMNN.oysosmMM/.yMMMyyo`dMMyohMMo .MMNNNN+NMN. -MMd             `dMMMs  oMMMMMMMMMMMMMm              ./ymMMMMm`    #
#    MMMhyyyy` mMM/    dMMo-MMM`             sMMy      .+syhmMMs -MMh   NMM.hMM/  `MMM`/MMh    .+syhmMMs `MMN   sMMo   mMM-.MMm`   :MMh dMN.              `dMMMs `mMMMMMMMMMMMM:          `/ymMMMMMms/`     #
#    MMM.      mMM/  `oMMm` mMMy`    /.      sMMy     :MMd:-oMMs -MMh  `NMM`hMM/  `MMN`/MMy   :MMd:-oMMs `MMN   oMMs   mMM-.MMm     oMMdMM:                `dMMMs `sNMMMMMMMMh-       `/smMMMMMms/`         #
#    MMM.      mMMmdmMMNs`  `yMMNdhdNM:      sMMNmmmmd-MMNssmMMs -MMNyyNMN/ .mMNysmMN/ /MMy   -MMNssmMMs  mMMhss`hMMysdMMo .MMm      hMMMs                  `dMMMs   -+ssso/`     `/smMMMMMms/`             #
#    ///`      //////:.       `:+oo+:.       -//////// .+o+:.//- `//::++/`    -+oo+:`  .//-    .+o+:.//-   :+o+:  ./ooo/`  `///      +MMd                    `dMMMs           ./smMMMMMms/`                 #
#                                                                                                                                   .NMN.                     `dMMMs      ./ymMMMMMms/`                     #
#                                                                                                                                   `..`                       `hMMMs `/smMMMMMmy/`                         #
#                                                                                                                                                               `hMMMmMMMMMmy/.                             #
#                                             T  H  E    M  O  T  I  O  N    T  E  C  H  N  O  L  O  G  Y    I  N  N  O  V  A  T  I  O  N  S                     `hMMMMmy/.                                 #
#                                                                                                                                                                 `yy/.                                     #
#                                                                                                                                                                                                           #
#   ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  #
#                                                                                                                                                                                                           #
#                    `++++++/-`    `+++++++-   -+ooo+:  .++.     ./oooo+-   +++`     :+/   +++++++-  -++++++++++++//:.                                                                                      #
#                    .MMmyyhmMMh-  `MMmyyyy: -mMmsosyy  /MM:   +mMmyssydh   MMMN:    hMm   NMmyyyy/  oMMMMMMMMMMMMMMMMMmy+.                                                                                 #
#                    .MMo    -mMN. `MMo      sMM/       /MM:  hMN/          MMhNMo   hMm   NMy       oMMMMMMMMMMMMMMMMMMMMMh:                                                                               #
#                    .MMo     :MMo `MMmyyyy` `yMMmy+.   /MM: :MMo  `+++++.  MMs.dMh` hMm   NMmyyyy.  oMMMMM+     `.:odMMMMMMMy                                                                              #
#                    .MMo     /MM+ `MMh++++`   `:sdMMy  /MM: :MMo  `yydMM:  MMs  sMm-hMm   NMd++++`  oMMMMM+          .yMMMMMMy                                                                             #
#                    .MMo   `+NMh  `MMo      .     sMM: /MM:  dMN/    /MM:  MMs   /NMmMm   NMy       oMMMMM+            +MMMMMM:                                                                            #
#                    .MMNddmMMh/   `MMNdddds sNdyydMMy  /MM:  `omMNdhhNMM-  MMs    .dMMm   NMNddddy  oMMMMM+             hMMMMMy                                                                            #
#                     :::::-.       :::::::- `-////-`   `::`     .:///:-`   ::.      ::-   :::::::-  oMMMMM+             +MMMMMm                                                                            #
#                                                                                                    oMMMMM+             +MMMMMd                                                                            #
#                                                                                .-`                 oMMMMM+             hMMMMMs                                                                            #
#                                                                                dM:                 oMMMMM+            /MMMMMM. `mmm.    /hmmmm+ `mmmmmm`  /hmNNmy:   dmd.   -mh                           #
#                                                                                dM:-//- `//`  `/:   oMMMMM+          `oMMMMMM/  hMoMd   -MM.   ` `MM`    `dMs.  -hMy  mMmN/  :Md                           #
#                                                                                dMdo+hMy sMs  yM/   oMMMMM+       `:sNMMMMMm:  oMs yMo   sNNh+.  `MMysso +Mm     .MM. mM-sMs :Md                           #
#                                                                                dM:  `MM` dM::Ms    oMMMMMmhhhhddNMMMMMMMNo`  :MMssyMM:    -omMy `MM:::- /MN`    -MM` mM- :Nd/Md                           #
#                                                                                dMy..sMd  `mmNd     oMMMMMMMMMMMMMMMMMmy:    `NM+:::+MN`-+-.-sMm `MM/:::` yMd+::oNN/  mM-  .dMMd                           #
#                                                                                os/shy/    :MN.     /hhhhhhhhhhhyso+:`       /s+     +s/.syhys+` `ssssss-  .+yhys/`   os.    oso                           #
#                                                                                         /+mN-                                                                                                             #
######################################################################################## /+:` ###############################################################################################################

#Env_landing.py

#ImportParty+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
import rospy
import threading
import numpy as np
import copy

from geometry_msgs.msg import PoseStamped, TwistStamped 
from sensor_msgs.msg import Imu, LaserScan, Image
from gazebo_msgs.msg import LinkState
from mavros_msgs.msg import RCIn, State 

#Initialization++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


#SubscriberCallbacks+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
def getLocalPose_Att(msg):                                                            

    global Lpose_x                                                                    
    global Lpose_y
    global Lpose_z

    global Att_x
    global Att_y
    global Att_z
    global Att_w

    global Stamp

    global PosAtt_flag

    Lpose_x = msg.pose.position.x                                                       
    Lpose_y = msg.pose.position.y                                                     
    Lpose_z = msg.pose.position.z

    Att_x   = msg.orientation.x                                                       
    Att_y   = msg.orientation.y                                                       
    Att_z   = msg.orientation.z      
    Att_w   = msg.orientation.w

    Stamp   = msg.header.secs + ( float( '0.' + str(msg.header.nsecs) ) )

    

def getLocalVel(msg):
    #not body-frame
    
    global Lvel_x                                                                    
    global Lvel_y
    global Lvel_z

    Lvel_x = msg.twist.linear.x
    Lvel_y = msg.twist.linear.y
    Lvel_z = msg.twist.linear.z

def getPixState(msg):
    global FMode

    Rawdat = msg.mode

    if Rawdat == 'Manual':
        k = 0
    elif Rawdat == 'Altitude':
        k = 1
    elif Rawdat == 'Position':
        k = 2
    else:
        k = 3
        pass

    FMode = k

def getVisual(msg):
    global raw_image
    global IMGresolution

    raw_image = msg.data
    IMGresolution = [ msg.width, msg.height ]

def get2DLidar(msg):
    global LaserDat

    LaserDat = msg.ranges

def getGDistance(msg):

def RCsignal(msg):
    global ThrottleCMD
    global YawRateCMD
    global RollCMD
    global PitchCMD
    global clearanceCMD

    ThrottleCMD     = msg.channels(2)
    YawRateCMD      = msg.channels(3)
    RollCMD         = msg.channels(0)
    PitchCMD        = msg.channels(1)
    clearanceCMD    = msg.channels(4)


#Functions++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
def OneWitch():
    PX4manipulation.pose.position.x     = 5.5
    PX4manipulation.pose.position.y     = -6.5
    PX4manipulation.pose.position.z     = 7.5

    PX4manipulation.pose.orientation.x  = 0  # yaw 수정
    PX4manipulation.pose.orientation.y  = 0
    PX4manipulation.pose.orientation.z  = 0
    PX4manipulation.pose.orientation.w  = 0

def VisualJoint(dat):
    global Vjoint
    Vjoint = copy.copy(dat.reshape(IMGresolution(1),IMGresolution(0),3))#.astype(np.uint8)#.reshape(480,640,3)

def MakeBNW(jointdat):
    sT4 = time.time()

    def do_work_square(te,be,le,re):
        global bnwviz
        for r in range(te,be):
        for c in range(le,re):
            bnwviz[r,c] = int ( float( jointdat[r,c,0] + jointdat[r,c,1] + jointdat[r,c,2] )/3 )
    num = 80
    threadss = []     

    for ber in range(num):
        thh = threading.Thread(target=do_work_square, args=((IMGresolution(1)/num)*ber,(IMGresolution(1)/num)*(ber+1),(IMGresolution(0)/num)*ber,(IMGresolution(0)/num)*(ber+1)))
        thh.start()
        threadss.append(thh)
    
    for thh in threadss:
        thh.join()
    print 'Dur : %f'%(time.time()-sT4)

    return bnwviz


#ROSNodeInitialization++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
rospy.init_node('OpenAIhawk')


#Manipulation-Initialization++++++++++++++++++++++++++++++++++++++++++++++++++++++++
PX4manipulation                     = LinkState()
PX4manipulation.link_name           = '링크이름'                                                                                                    #수정해야되.
OneWitch()

#Subscriber-Initialization++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
aquiz_px4_pose_att_dat  = rospy.Subscriber('/mavros/local_position/pose',           PoseStamped,    getLocalPose_Att    )
aquiz_px4_vel_dat       = rospy.Subscriber('/mavros/local_posion/velocity_local',   TwistStamped,   getLocalVel         )
aquiz_px4_state         = rospy.Subscriber('/mavros/state',                         State,          getPixState         )
aquiz_rplidar_dat       = rospy.Subscriber('/laser/scan',                           LaserScan,      get2DLidar          )
aquiz_camera_dat        = rospy.Subscriber('/iris_fpv_cam/usb_cam/image_raw',       Image,          getVisual           )
rate = rospy.Rate(30)

#ItStarts+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
while ~rospy.is_shutdown():




rospy.spin()


# Landing data aquisition

In [ ]:
#!/usr/bin/env python
#-*- coding: utf-8 -*-
                                                       
################################################################################################################################################################################ -ohh`  #####################                
#                                                                                                                                                                             -ohNMMMMd`                    #
#                                                                                                                                                                         -ohNMMMMMmMMMd`                   #
#                                                                                                                                                                     -ohNMMMMNho-  yMMMd`                  #
#                                                                                                                                                                 -ohNMMMMNho-       yMMMd`                 #
#                                                                                                                                                             -ohNMMMMNho-            yMMMd`                #
#                                                                                                                                                         -+hNMMMMMdo-                 yMMMd`               #
#                                                                                                                                                     -+hNMMMMMdo:`                     yMMMd`              #
#                                                                                                                                                 -+hNMMMMMho-                           sMMMd`             #
#                                                                                                                                                `dMMMMho-                                sMMMd`            #
#                                                                                                                                                 `dMMMo                                   sMMMd`           #
#                                                                                                                                                  `dMMMs                                   sMMMd`          #
#                                                                                                                                                   `dMMMs                                   sMMMm`         #
#                                                                                                                                                    `dMMMo         `:osyyo/.                 sMMMm`        #
#                                                                                                                                                     `dMMMo      `sNMMMMMMMMh-                sMMMm`       #
#    MMMMMMMMo mMMMMNNds-    -yNMMMMNd-      sMMs                -MMh                                    `ddh                                          `dMMMo    `mMMMMMMMMMMMM/                sMMMm`      #
#    MMM/----. mMMo--+mMMy  oMMN+.`./h:      sMMy      -+osso/.  -MMh-+so:    :osso/`  .++:`/+`-+osso/. `+MMN++/  -+sso/.  `++/`:+/++-   :++.           `dMMMo   oMMMMMMMMMMMMMm                 sMMMm.     #
#    MMMyssss` mMM/   `mMM/.MMM-             sMMy      oysosmMM/ -MMNhydMMy -mMNsodMN/ /MMNMNN.oysosmMM/.yMMMyyo`dMMyohMMo .MMNNNN+NMN. -MMd             `dMMMs  oMMMMMMMMMMMMMm              ./ymMMMMm`    #
#    MMMhyyyy` mMM/    dMMo-MMM`             sMMy      .+syhmMMs -MMh   NMM.hMM/  `MMM`/MMh    .+syhmMMs `MMN   sMMo   mMM-.MMm`   :MMh dMN.              `dMMMs `mMMMMMMMMMMMM:          `/ymMMMMMms/`     #
#    MMM.      mMM/  `oMMm` mMMy`    /.      sMMy     :MMd:-oMMs -MMh  `NMM`hMM/  `MMN`/MMy   :MMd:-oMMs `MMN   oMMs   mMM-.MMm     oMMdMM:                `dMMMs `sNMMMMMMMMh-       `/smMMMMMms/`         #
#    MMM.      mMMmdmMMNs`  `yMMNdhdNM:      sMMNmmmmd-MMNssmMMs -MMNyyNMN/ .mMNysmMN/ /MMy   -MMNssmMMs  mMMhss`hMMysdMMo .MMm      hMMMs                  `dMMMs   -+ssso/`     `/smMMMMMms/`             #
#    ///`      //////:.       `:+oo+:.       -//////// .+o+:.//- `//::++/`    -+oo+:`  .//-    .+o+:.//-   :+o+:  ./ooo/`  `///      +MMd                    `dMMMs           ./smMMMMMms/`                 #
#                                                                                                                                   .NMN.                     `dMMMs      ./ymMMMMMms/`                     #
#                                                                                                                                   `..`                       `hMMMs `/smMMMMMmy/`                         #
#                                                                                                                                                               `hMMMmMMMMMmy/.                             #
#                                             T  H  E    M  O  T  I  O  N    T  E  C  H  N  O  L  O  G  Y    I  N  N  O  V  A  T  I  O  N  S                     `hMMMMmy/.                                 #
#                                                                                                                                                                 `yy/.                                     #
#                                                                                                                                                                                                           #
#   ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  #
#                                                                                                                                                                                                           #
#                    `++++++/-`    `+++++++-   -+ooo+:  .++.     ./oooo+-   +++`     :+/   +++++++-  -++++++++++++//:.                                                                                      #
#                    .MMmyyhmMMh-  `MMmyyyy: -mMmsosyy  /MM:   +mMmyssydh   MMMN:    hMm   NMmyyyy/  oMMMMMMMMMMMMMMMMMmy+.                                                                                 #
#                    .MMo    -mMN. `MMo      sMM/       /MM:  hMN/          MMhNMo   hMm   NMy       oMMMMMMMMMMMMMMMMMMMMMh:                                                                               #
#                    .MMo     :MMo `MMmyyyy` `yMMmy+.   /MM: :MMo  `+++++.  MMs.dMh` hMm   NMmyyyy.  oMMMMM+     `.:odMMMMMMMy                                                                              #
#                    .MMo     /MM+ `MMh++++`   `:sdMMy  /MM: :MMo  `yydMM:  MMs  sMm-hMm   NMd++++`  oMMMMM+          .yMMMMMMy                                                                             #
#                    .MMo   `+NMh  `MMo      .     sMM: /MM:  dMN/    /MM:  MMs   /NMmMm   NMy       oMMMMM+            +MMMMMM:                                                                            #
#                    .MMNddmMMh/   `MMNdddds sNdyydMMy  /MM:  `omMNdhhNMM-  MMs    .dMMm   NMNddddy  oMMMMM+             hMMMMMy                                                                            #
#                     :::::-.       :::::::- `-////-`   `::`     .:///:-`   ::.      ::-   :::::::-  oMMMMM+             +MMMMMm                                                                            #
#                                                                                                    oMMMMM+             +MMMMMd                                                                            #
#                                                                                .-`                 oMMMMM+             hMMMMMs                                                                            #
#                                                                                dM:                 oMMMMM+            /MMMMMM. `mmm.    /hmmmm+ `mmmmmm`  /hmNNmy:   dmd.   -mh                           #
#                                                                                dM:-//- `//`  `/:   oMMMMM+          `oMMMMMM/  hMoMd   -MM.   ` `MM`    `dMs.  -hMy  mMmN/  :Md                           #
#                                                                                dMdo+hMy sMs  yM/   oMMMMM+       `:sNMMMMMm:  oMs yMo   sNNh+.  `MMysso +Mm     .MM. mM-sMs :Md                           #
#                                                                                dM:  `MM` dM::Ms    oMMMMMmhhhhddNMMMMMMMNo`  :MMssyMM:    -omMy `MM:::- /MN`    -MM` mM- :Nd/Md                           #
#                                                                                dMy..sMd  `mmNd     oMMMMMMMMMMMMMMMMMmy:    `NM+:::+MN`-+-.-sMm `MM/:::` yMd+::oNN/  mM-  .dMMd                           #
#                                                                                os/shy/    :MN.     /hhhhhhhhhhhyso+:`       /s+     +s/.syhys+` `ssssss-  .+yhys/`   os.    oso                           #
#                                                                                         /+mN-                                                                                                             #
######################################################################################## /+:` ###############################################################################################################

#OpenAI_hawk.py

#ImportParty+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
import rospy
import threading
import numpy as np
import copy
import json
import time

from geometry_msgs.msg import PoseStamped, TwistStamped 
from sensor_msgs.msg import Imu, LaserScan, Image
from gazebo_msgs.msg import LinkState, ModelState
from mavros_msgs.msg import RCIn, State 

#Setups+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
fileDerectory = "/home/daseonbig16/Documents/LandingData/"

#Variable Initialization++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
isstartkey  = False
isstopkey   = False

#dictionary
LandingDim      = {'centerCoord':[0.04,0.18,0.03], 'width':0.5, 'height':0.5}
IMGresolution   = {'width':320, 'height':240}
ScanInfo        = {'num_of_dots':1440}

#Global Variable Initialization
Lpose_x         = 0.                                                                    
Lpose_y         = 0.
Lpose_z         = 0.
Lvel_x          = 0.0
Lvel_y          = 0.0
Lvel_z          = 0.0
Att_x           = 0.
Att_y           = 0.
Att_z           = 0.
Att_w           = 0.
Stamp           = 0.
PosAtt_flag     = 0
FMode           = 0
raw_image       = np.zeros( IMGresolution['width'] * IMGresolution['height'] * 3, dtype = np.uint8 )
ThrottleCMD     = 0
YawRateCMD      = 0
RollCMD         = 0
PitchCMD        = 0
clearanceCMD    = 9999
prevstamp       = 0
Stamp_secs      = 0
Stamp_nsecs     = 0
WholeSaler      = {}
thisrunSaler    = {}
thisframeSaler  = {}
sequenceCounter = 0
frameCounter    = 0
savedflag       = False
initializeflag  = False
firstdataget    = False 
view1           = np.zeros( IMGresolution['width'] * IMGresolution['height'] * 3, dtype = np.uint8 )
view2           = np.zeros( IMGresolution['width'] * IMGresolution['height'] * 3, dtype = np.uint8 )
view3           = np.zeros( IMGresolution['width'] * IMGresolution['height'] * 3, dtype = np.uint8 )
GSviz           = np.zeros( [IMGresolution['height'] , IMGresolution['width']], dtype = np.uint8 )
clearanceCMD    = 1999

#SubscriberCallbacks+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
def getLocalPose_Att(msg):                                                            

    global Lpose_x                                                                    
    global Lpose_y
    global Lpose_z

    global Att_x
    global Att_y
    global Att_z
    global Att_w

    global Stamp

    global PosAtt_flag

    Lpose_x     = msg.pose.position.x                                                       
    Lpose_y     = msg.pose.position.y                                                     
    Lpose_z     = msg.pose.position.z

    Att_x       = msg.pose.orientation.x                                                       
    Att_y       = msg.pose.orientation.y                                                       
    Att_z       = msg.pose.orientation.z      
    Att_w       = msg.pose.orientation.w
    
    Stamp       = msg.header.stamp.secs + ( float( '0.' + str(msg.header.stamp.nsecs) ) )
    Stamp_secs  = msg.header.stamp.secs
    Stamp_nsecs = msg.header.stamp.nsecs 

def getLocalVel(msg):
    #not body-frame
    
    global Lvel_x                                                                    
    global Lvel_y
    global Lvel_z

    Lvel_x    = msg.twist.linear.x
    Lvel_y    = msg.twist.linear.y
    Lvel_z    = msg.twist.linear.z

def getPixState(msg):
    global FMode

    Rawdat    = msg.mode

    if Rawdat   == 'Manual':
        k = 0
    elif Rawdat == 'Altitude':
        k = 1
    elif Rawdat == 'Position':
        k = 2
    else:
        k = 3
        pass

    FMode = k

def getRCsignal(msg):
    global ThrottleCMD
    global YawRateCMD
    global RollCMD
    global PitchCMD
    global clearanceCMD

    ThrottleCMD     = msg.channels[2]
    YawRateCMD      = msg.channels[3]
    RollCMD         = msg.channels[0]
    PitchCMD        = msg.channels[1]
    clearanceCMD    = msg.channels[4]

def getVisual1(msg):
    global view1
    #lili = (msg.data)
    sT4 = time.time()
    view1 = np.array([ord(i) for i in msg.data])
    print 'data conversion dur : %f'%(time.time()-sT4)
    
def getVisual2(msg):
    global view2
    #lili = (msg.data)
    view2 = np.array([ord(i) for i in msg.data])
    
def getVisual3(msg):
    global view3
    #lili = (msg.data)
    view3 = np.array([ord(i) for i in msg.data])

#Functions++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
def OneWitch():
    global PX4manipulation
    PX4manipulation.pose.position.x     = 57.15
    PX4manipulation.pose.position.y     = -21.881
    PX4manipulation.pose.position.z     = 25.0

    PX4manipulation.pose.orientation.x  = 0.00067135 
    PX4manipulation.pose.orientation.y  = -0.0026451
    PX4manipulation.pose.orientation.z  = 0.9935357
    PX4manipulation.pose.orientation.w  = 0.1135184
      
def OneWitchSet():                      # 쓰레기
    global PX4_witch_set
    PX4_witch_set.header.stamp.secs     = Stamp_secs
    PX4_witch_set.header.stamp.nsecs    = Stamp_nsecs
    PX4_witch_set.header.frame_id       = 'map'
      
    PX4_witch_set.pose.position.x       = 57.15
    PX4_witch_set.pose.position.y       = -21.881
    PX4_witch_set.pose.position.z       = 25.0
      
    PX4_witch_set.pose.orientation.x    = 0.00067135 
    PX4_witch_set.pose.orientation.y    = -0.0026451
    PX4_witch_set.pose.orientation.z    = 0.9935357
    PX4_witch_set.pose.orientation.w    = 0.1135184
      
def isitLandedQuestionmark(curpose):
    if (( curpose[0] <= LandingDim['centerCoord'][0] + LandingDim['width'] ) & ( curpose[0] >= LandingDim['centerCoord'][0] - LandingDim['width'] )\
       &( curpose[1] <= LandingDim['centerCoord'][1] + LandingDim['width'] ) & ( curpose[1] >= LandingDim['centerCoord'][1] - LandingDim['width'] )\
       &( curpose[2] <= LandingDim['centerCoord'][2] + LandingDim['height']) & ( curpose[2] >= LandingDim['centerCoord'][2] - LandingDim['height'])):
        return True
    else:
        return False
      
def VisualHexahedralize(dat):           # Compile the raw data into hexahedral matrix
    VHexahedron = copy.copy(dat.reshape(IMGresolution['height'],IMGresolution['width'],3).astype(np.uint8))#reshape(480,640,3)
    return VHexahedron
      
def MakeGS(jointdat):                   # RGB8 to Greyscale converter
    #sT4 = time.time()

    def do_work_square(te,be,le,re):
        global GSviz
        #print te,be,le,re
        for r in range(te,be):
            for c in range(le,re):
                GSviz[r,c] = np.uint8 ( float( np.uint16(jointdat[r,c,0]) + np.uint16(jointdat[r,c,1]) + np.uint16(jointdat[r,c,2]) )/3 )
    slicenum = 1
    threadss = []     

    for ber in range(slicenum):
        for num in range(slicenum):
            thh = threading.Thread(target=do_work_square, \
                args=((IMGresolution['height']/slicenum)*ber,(IMGresolution['height']/slicenum)*(ber+1),(IMGresolution['width']/slicenum)*num,(IMGresolution['width']/slicenum)*(num+1)))
            thh.start()
            threadss.append(thh)
    
    for thh in threadss:
        thh.join()
    #print 'Dur : %f'%(time.time()-sT4)

    return GSviz
      
def gatherData_thisframe(stamp, views, velvec):
    global frameCounter
    sT6 = time.time()
    thisframeSaler  = {"stamp":stamp, "cam":views, "velvec":velvec}
    frameCounter    = frameCounter + 1
    print 'put into the dic dur : %f'%(time.time()-sT6)
    return thisframeSaler
      
def gatherData_thisrun(frameSaler):
    #global frameCounter
    thisrunSaler.update( {frameCounter:frameSaler} )

    #frameCounter    = frameCounter + 1
    return thisrunSaler

def gatherData_Whole(runSaler):
    global sequenceCounter    
    WholeSaler.update( {sequenceCounter:runSaler} )

    sequenceCounter = sequenceCounter + 1
    return WholeSaler 

def save_jason(data):
    sT5 = time.time()
    jstring = json.dumps(data, indent=4)
    filename = fileDerectory + 'landingdat' + '_' + '%d'%sequenceCounter + '_' + '%d'%frameCounter + '.json'
    f = open(filename, "w")
    f.write(jstring)
    f.close()
    print 'saving dur : %f'%(time.time()-sT5)

def jasonSerializablize(numpyarray):
    listizedArray = numpyarray.tolist()
    return listizedArray

#ROSNodeInitialization++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
rospy.init_node('OpenAIhawk')


#Manipulation-Initialization++++++++++++++++++++++++++++++++++++++++++++++++++++++++
PX4manipulation                     = ModelState()
PX4manipulation.model_name          = 'iris_triplecam_env'                       #수정해야되.
OneWitch()

#PX4_witch_set                       = PoseStamped()
#OneWitchSet()

#Subscriber-Initialization++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
aquiz_px4_pose_att_dat  = rospy.Subscriber('/mavros/local_position/pose',               PoseStamped,    getLocalPose_Att    )
aquiz_px4_vel_dat       = rospy.Subscriber('/mavros/local_position/velocity_local',     TwistStamped,   getLocalVel         )
aquiz_px4_state         = rospy.Subscriber('/mavros/state',                             State,          getPixState         )
aquiz_px4_RC            = rospy.Subscriber('/mavros/rc/in',                             RCIn,           getRCsignal         ) 

aquiz_camera_dat1        = rospy.Subscriber('/iris_triplecam_env/usb_cam/image_raw',    Image,          getVisual1          ) 
aquiz_camera_dat2        = rospy.Subscriber('/iris_triplecam_env/usb_cam_2/image_raw',  Image,          getVisual2          ) 
aquiz_camera_dat3        = rospy.Subscriber('/iris_triplecam_env/usb_cam_3/image_raw',  Image,          getVisual3          ) 
                                                                                                                            
#Publisher-Initialization+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++                    
cmd_cube_pub            = rospy.Publisher('/gazebo/set_model_state',                    ModelState,     queue_size=1        )
cmd_set_pos             = rospy.Publisher('/mavros/setpoint_position/local',            PoseStamped,    queue_size=1        )
rate = rospy.Rate(30)

#++++++++++++++++++++++++++++++Initialization Ends++++++++++++++++++++++++++++++++++
prevstamp = 0
# cmd_cube_pub.publish(PX4manipulation)
#ItStarts+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
while ~rospy.is_shutdown():

    if (clearanceCMD > 1900) & (firstdataget == False) :
        
        while (clearanceCMD > 1900):
            print 'pull down the trigger so it is on the second phase'
        pass

    elif (clearanceCMD > 1900) & (firstdataget == True): 
        print 'Cessation command detected, byebye'
        break

    curpose = [Lpose_x, Lpose_y, Lpose_z]
    # gather data of the frame and append'em into the thisrun data set
    '''
    gatheredRun =  gatherData_thisrun(\
                                gatherData_thisframe(   Stamp, \
                                                        {"cam1":jasonSerializablize( MakeGS( VisualHexahedralize( view1 ) )), \
                                                         "cam2":jasonSerializablize( MakeGS( VisualHexahedralize( view2 ) )), \
                                                         "cam3":jasonSerializablize( MakeGS( VisualHexahedralize( view3 ) ))},\
                                                        [ Lvel_x, Lvel_y, Lvel_z ] ) \
    ) 
    '''
    print Lvel_x
    if (savedflag == False) & (clearanceCMD > 1490):
            print 'saving'
            #updated = gatherData_Whole( gatheredRun )
            #save_jason( updated )
            
            save_jason( gatherData_thisframe( Stamp, \
                                             {"cam1":jasonSerializablize( MakeGS( VisualHexahedralize( view1 ) )), \
                                              "cam2":jasonSerializablize( MakeGS( VisualHexahedralize( view2 ) )), \
                                              "cam3":jasonSerializablize( MakeGS( VisualHexahedralize( view3 ) ))},\
                                              [ Lvel_x, Lvel_y, Lvel_z ] ) )
            savedflag = True
            if firstdataget == False : 
                firstdataget = True
                clearanceCMD = 1899
            
            print 'saved' + '_' + '%d'%sequenceCounter + '_' + '%d'%frameCounter

    
    
    if (clearanceCMD <=1490) :
        
        OneWitch()
        frameCounter = 0
        cmd_cube_pub.publish(PX4manipulation)
        
        if initializeflag == False:
            sequenceCounter = sequenceCounter + 1
            
            initializeflag = True

        print 'initialization command is detected, switch the trigger to the middle'

    
    else:
        initializeflag = False
        print 'new sequence started'

    if( (isitLandedQuestionmark(curpose) == True) ):
        print 'landed properly' #는 local position이 순간이동하면 최신화가 안되기 때문에 뜨기 힘듬
    print 'CMD', clearanceCMD
    savedflag = False
    
    
quit()



# Landing Data Acquisition 2 #

In [ ]:
#!/usr/bin/env python
#-*- coding: utf-8 -*-
                                                       
################################################################################################################################################################################ -ohh`  #####################                
#                                                                                                                                                                             -ohNMMMMd`                    #
#                                                                                                                                                                         -ohNMMMMMmMMMd`                   #
#                                                                                                                                                                     -ohNMMMMNho-  yMMMd`                  #
#                                                                                                                                                                 -ohNMMMMNho-       yMMMd`                 #
#                                                                                                                                                             -ohNMMMMNho-            yMMMd`                #
#                                                                                                                                                         -+hNMMMMMdo-                 yMMMd`               #
#                                                                                                                                                     -+hNMMMMMdo:`                     yMMMd`              #
#                                                                                                                                                 -+hNMMMMMho-                           sMMMd`             #
#                                                                                                                                                `dMMMMho-                                sMMMd`            #
#                                                                                                                                                 `dMMMo                                   sMMMd`           #
#                                                                                                                                                  `dMMMs                                   sMMMd`          #
#                                                                                                                                                   `dMMMs                                   sMMMm`         #
#                                                                                                                                                    `dMMMo         `:osyyo/.                 sMMMm`        #
#                                                                                                                                                     `dMMMo      `sNMMMMMMMMh-                sMMMm`       #
#    MMMMMMMMo mMMMMNNds-    -yNMMMMNd-      sMMs                -MMh                                    `ddh                                          `dMMMo    `mMMMMMMMMMMMM/                sMMMm`      #
#    MMM/----. mMMo--+mMMy  oMMN+.`./h:      sMMy      -+osso/.  -MMh-+so:    :osso/`  .++:`/+`-+osso/. `+MMN++/  -+sso/.  `++/`:+/++-   :++.           `dMMMo   oMMMMMMMMMMMMMm                 sMMMm.     #
#    MMMyssss` mMM/   `mMM/.MMM-             sMMy      oysosmMM/ -MMNhydMMy -mMNsodMN/ /MMNMNN.oysosmMM/.yMMMyyo`dMMyohMMo .MMNNNN+NMN. -MMd             `dMMMs  oMMMMMMMMMMMMMm              ./ymMMMMm`    #
#    MMMhyyyy` mMM/    dMMo-MMM`             sMMy      .+syhmMMs -MMh   NMM.hMM/  `MMM`/MMh    .+syhmMMs `MMN   sMMo   mMM-.MMm`   :MMh dMN.              `dMMMs `mMMMMMMMMMMMM:          `/ymMMMMMms/`     #
#    MMM.      mMM/  `oMMm` mMMy`    /.      sMMy     :MMd:-oMMs -MMh  `NMM`hMM/  `MMN`/MMy   :MMd:-oMMs `MMN   oMMs   mMM-.MMm     oMMdMM:                `dMMMs `sNMMMMMMMMh-       `/smMMMMMms/`         #
#    MMM.      mMMmdmMMNs`  `yMMNdhdNM:      sMMNmmmmd-MMNssmMMs -MMNyyNMN/ .mMNysmMN/ /MMy   -MMNssmMMs  mMMhss`hMMysdMMo .MMm      hMMMs                  `dMMMs   -+ssso/`     `/smMMMMMms/`             #
#    ///`      //////:.       `:+oo+:.       -//////// .+o+:.//- `//::++/`    -+oo+:`  .//-    .+o+:.//-   :+o+:  ./ooo/`  `///      +MMd                    `dMMMs           ./smMMMMMms/`                 #
#                                                                                                                                   .NMN.                     `dMMMs      ./ymMMMMMms/`                     #
#                                                                                                                                   `..`                       `hMMMs `/smMMMMMmy/`                         #
#                                                                                                                                                               `hMMMmMMMMMmy/.                             #
#                                             T  H  E    M  O  T  I  O  N    T  E  C  H  N  O  L  O  G  Y    I  N  N  O  V  A  T  I  O  N  S                     `hMMMMmy/.                                 #
#                                                                                                                                                                 `yy/.                                     #
#                                                                                                                                                                                                           #
#   ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  #
#                                                                                                                                                                                                           #
#                    `++++++/-`    `+++++++-   -+ooo+:  .++.     ./oooo+-   +++`     :+/   +++++++-  -++++++++++++//:.                                                                                      #
#                    .MMmyyhmMMh-  `MMmyyyy: -mMmsosyy  /MM:   +mMmyssydh   MMMN:    hMm   NMmyyyy/  oMMMMMMMMMMMMMMMMMmy+.                                                                                 #
#                    .MMo    -mMN. `MMo      sMM/       /MM:  hMN/          MMhNMo   hMm   NMy       oMMMMMMMMMMMMMMMMMMMMMh:                                                                               #
#                    .MMo     :MMo `MMmyyyy` `yMMmy+.   /MM: :MMo  `+++++.  MMs.dMh` hMm   NMmyyyy.  oMMMMM+     `.:odMMMMMMMy                                                                              #
#                    .MMo     /MM+ `MMh++++`   `:sdMMy  /MM: :MMo  `yydMM:  MMs  sMm-hMm   NMd++++`  oMMMMM+          .yMMMMMMy                                                                             #
#                    .MMo   `+NMh  `MMo      .     sMM: /MM:  dMN/    /MM:  MMs   /NMmMm   NMy       oMMMMM+            +MMMMMM:                                                                            #
#                    .MMNddmMMh/   `MMNdddds sNdyydMMy  /MM:  `omMNdhhNMM-  MMs    .dMMm   NMNddddy  oMMMMM+             hMMMMMy                                                                            #
#                     :::::-.       :::::::- `-////-`   `::`     .:///:-`   ::.      ::-   :::::::-  oMMMMM+             +MMMMMm                                                                            #
#                                                                                                    oMMMMM+             +MMMMMd                                                                            #
#                                                                                .-`                 oMMMMM+             hMMMMMs                                                                            #
#                                                                                dM:                 oMMMMM+            /MMMMMM. `mmm.    /hmmmm+ `mmmmmm`  /hmNNmy:   dmd.   -mh                           #
#                                                                                dM:-//- `//`  `/:   oMMMMM+          `oMMMMMM/  hMoMd   -MM.   ` `MM`    `dMs.  -hMy  mMmN/  :Md                           #
#                                                                                dMdo+hMy sMs  yM/   oMMMMM+       `:sNMMMMMm:  oMs yMo   sNNh+.  `MMysso +Mm     .MM. mM-sMs :Md                           #
#                                                                                dM:  `MM` dM::Ms    oMMMMMmhhhhddNMMMMMMMNo`  :MMssyMM:    -omMy `MM:::- /MN`    -MM` mM- :Nd/Md                           #
#                                                                                dMy..sMd  `mmNd     oMMMMMMMMMMMMMMMMMmy:    `NM+:::+MN`-+-.-sMm `MM/:::` yMd+::oNN/  mM-  .dMMd                           #
#                                                                                os/shy/    :MN.     /hhhhhhhhhhhyso+:`       /s+     +s/.syhys+` `ssssss-  .+yhys/`   os.    oso                           #
#                                                                                         /+mN-                                                                                                             #
######################################################################################## /+:` ###############################################################################################################

#OpenAI_hawk version 1.2
#   what's new
#       - NN friendly type of csv file system
#       - Better stability
#       - Various initial point system is availiable
#       - No teleportation

#ImportParty+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
import rospy
import threading
import numpy as np
import copy
#import json
import time
import csv
import sys

from geometry_msgs.msg import PoseStamped, TwistStamped 
from sensor_msgs.msg import Imu, LaserScan, Image
from gazebo_msgs.msg import LinkState, ModelState
from mavros_msgs.msg import RCIn, State 

#Setups+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
fileDerectory = "/home/daseonbig16/Documents/LandingData/"
#csv.field_size_limit(sys.maxsize)
#Variable Initialization++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
isstartkey  = False
isstopkey   = False

#dictionary
LandingDim      = {'centerCoord':[0.04,0.18,0.03], 'width':0.5, 'height':0.5}
IMGresolution   = {'width':320, 'height':240}
ScanInfo        = {'num_of_dots':1440}

#Global Variable Initialization
Lpose_x         = 0.                                                                    
Lpose_y         = 0.
Lpose_z         = 0.
Lvel_x          = 0.0
Lvel_y          = 0.0
Lvel_z          = 0.0
Att_x           = 0.
Att_y           = 0.
Att_z           = 0.
Att_w           = 0.
Stamp           = 0.
PosAtt_flag     = 0
FMode           = 0
raw_image       = np.zeros( IMGresolution['width'] * IMGresolution['height'] * 3, dtype = np.uint8 )
ThrottleCMD     = 0
YawRateCMD      = 0
RollCMD         = 0
PitchCMD        = 0
clearanceCMD    = 9999
prevstamp       = 0
Stamp_secs      = 0
Stamp_nsecs     = 0
WholeSaler      = {}
thisrunSaler    = []
thisframeSaler  = []
sequenceCounter = 0
frameCounter    = 0
runCounter      = 0
savedflag       = False
initializeflag  = False
firstdataget    = False 
view1           = np.zeros( IMGresolution['width'] * IMGresolution['height'] * 3, dtype = np.uint8 )
view2           = np.zeros( IMGresolution['width'] * IMGresolution['height'] * 3, dtype = np.uint8 )
view3           = np.zeros( IMGresolution['width'] * IMGresolution['height'] * 3, dtype = np.uint8 )
GSviz           = np.zeros( [IMGresolution['height'] , IMGresolution['width']], dtype = np.uint8 )
clearanceCMD    = 1999

#SubscriberCallbacks+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
def getLocalPose_Att(msg):                                                            

    global Lpose_x                                                                    
    global Lpose_y
    global Lpose_z

    global Att_x
    global Att_y
    global Att_z
    global Att_w

    global Stamp

    global PosAtt_flag

    Lpose_x = msg.pose.position.x                                                       
    Lpose_y = msg.pose.position.y                                                     
    Lpose_z = msg.pose.position.z

    Att_x   = msg.pose.orientation.x                                                       
    Att_y   = msg.pose.orientation.y                                                       
    Att_z   = msg.pose.orientation.z      
    Att_w   = msg.pose.orientation.w
    
    Stamp   = msg.header.stamp.secs + ( float( '0.' + str(msg.header.stamp.nsecs) ) )
    Stamp_secs  = msg.header.stamp.secs
    Stamp_nsecs = msg.header.stamp.nsecs 

def getLocalVel(msg):
    #not body-frame
    
    global Lvel_x                                                                    
    global Lvel_y
    global Lvel_z

    Lvel_x = msg.twist.linear.x
    Lvel_y = msg.twist.linear.y
    Lvel_z = msg.twist.linear.z

def getPixState(msg):
    global FMode

    Rawdat = msg.mode

    if Rawdat == 'Manual':
        k = 0
    elif Rawdat == 'Altitude':
        k = 1
    elif Rawdat == 'Position':
        k = 2
    else:
        k = 3
        pass

    FMode = k

def getRCsignal(msg):
    global ThrottleCMD
    global YawRateCMD
    global RollCMD
    global PitchCMD
    global clearanceCMD

    ThrottleCMD     = msg.channels[2]
    YawRateCMD      = msg.channels[3]
    RollCMD         = msg.channels[0]
    PitchCMD        = msg.channels[1]
    clearanceCMD    = msg.channels[4]

def getVisual1(msg):
    global view1
    #lili = (msg.data)
    #
    view1 = np.fromstring(msg.data, np.uint8)
    #print 'data conversion dur : %f'%(time.time()-sT4)
    
def getVisual2(msg):
    global view2
    #lili = (msg.data)
    view2 = np.fromstring(msg.data, np.uint8)
    
def getVisual3(msg):
    global view3
    #lili = (msg.data)
    view3 = np.fromstring(msg.data, np.uint8)
    #view3 = np.array([ord(i) for i in msg.data])

#Functions++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
def OneWitch():
    global PX4manipulation
    PX4manipulation.pose.position.x     = 57.15
    PX4manipulation.pose.position.y     = -21.881
    PX4manipulation.pose.position.z     = 25.0

    PX4manipulation.pose.orientation.x  = 0.00067135 
    PX4manipulation.pose.orientation.y  = -0.0026451
    PX4manipulation.pose.orientation.z  = 0.9935357
    PX4manipulation.pose.orientation.w  = 0.1135184
      
def OneWitchSet():                      # 쓰레기
    global PX4_witch_set
    PX4_witch_set.header.stamp.secs     = Stamp_secs
    PX4_witch_set.header.stamp.nsecs    = Stamp_nsecs
    PX4_witch_set.header.frame_id       = 'map'
      
    PX4_witch_set.pose.position.x       = 57.15
    PX4_witch_set.pose.position.y       = -21.881
    PX4_witch_set.pose.position.z       = 25.0
      
    PX4_witch_set.pose.orientation.x    = 0.00067135 
    PX4_witch_set.pose.orientation.y    = -0.0026451
    PX4_witch_set.pose.orientation.z    = 0.9935357
    PX4_witch_set.pose.orientation.w    = 0.1135184
      
def isitLandedQuestionmark(curpose):
    if (( curpose[0] <= LandingDim['centerCoord'][0] + LandingDim['width'] ) & ( curpose[0] >= LandingDim['centerCoord'][0] - LandingDim['width'] )\
       &( curpose[1] <= LandingDim['centerCoord'][1] + LandingDim['width'] ) & ( curpose[1] >= LandingDim['centerCoord'][1] - LandingDim['width'] )\
       &( curpose[2] <= LandingDim['centerCoord'][2] + LandingDim['height']) & ( curpose[2] >= LandingDim['centerCoord'][2] - LandingDim['height'])):
        return True
    else:
        return False
      
def VisualHexahedralize(dat):           # Compile the raw data into hexahedral matrix
    VHexahedron = copy.copy(dat.reshape(IMGresolution['height'],IMGresolution['width'],3).astype(np.uint8))#reshape(480,640,3)
    return VHexahedron

def MakeGS(jointdat):
    
    GSviz = (jointdat[:,:,0] + jointdat[:,:,1] + jointdat[:,:,2])/3
    
    return GSviz

def gatherData_thisframe(stamp, views, velvec):
    #thisframeSaler                  = []
    
    thisframeSaler[0:76800]         = views[0][:]

    thisframeSaler[76800:153600]    = views[1][:]

    thisframeSaler[153600:230400]   = views[2][:]

    thisframeSaler[230400:230403]   = velvec
    
    return thisframeSaler
      
def gatherData_thisrun(frameSaler,frame=0):
    global frameCounter
    global thisrunSaler    
    if frameSaler == 'bringME':
        return thisrunSaler[frame][:]

    elif frameSaler == 'clearCMD':
        thisrunSaler = []
        frameCounter = 0
        print 'clearRAM'

    else:
        thisrunSaler.append(frameSaler)
        frameCounter    = frameCounter + 1

def save_csv(frame,data=0):
    global filepointer
    global author
    if frame == 'open':
        filename = fileDerectory + 'dat' + '.csv'
        filepointer = open(filename, 'a')#, encoding='utf-8')
        author      = csv.writer(filepointer)
        print 'passed'
    elif frame == 'close':
        filepointer.close()
    
    else:
        sT5 = time.time()
        author.writerow(data)
        print 'saving dur : %f'%(time.time()-sT5)
    
def csvSerializablize(numpyarray):
    
    listizednumpy   = numpyarray.tolist()
    oneDimArray     = [y for x in listizednumpy for y in x]
    #oneDimArray     = sum(listizednumpy, [])

    return oneDimArray

#ROSNodeInitialization++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
rospy.init_node('OpenAIhawk')


#Manipulation-Initialization++++++++++++++++++++++++++++++++++++++++++++++++++++++++
PX4manipulation                     = ModelState()
PX4manipulation.model_name          = 'iris_triplecam_env'                       #수정해야되.
OneWitch()

#PX4_witch_set                       = PoseStamped()
#OneWitchSet()

#Subscriber-Initialization++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
aquiz_px4_pose_att_dat  = rospy.Subscriber('/mavros/local_position/pose',               PoseStamped,    getLocalPose_Att    )
aquiz_px4_vel_dat       = rospy.Subscriber('/mavros/local_position/velocity_local',     TwistStamped,   getLocalVel         )
aquiz_px4_state         = rospy.Subscriber('/mavros/state',                             State,          getPixState         )
aquiz_px4_RC            = rospy.Subscriber('/mavros/rc/in',                             RCIn,           getRCsignal         ) 

aquiz_camera_dat1        = rospy.Subscriber('/iris_triplecam_env/usb_cam/image_raw',    Image,          getVisual1          ) 
aquiz_camera_dat2        = rospy.Subscriber('/iris_triplecam_env/usb_cam_2/image_raw',  Image,          getVisual2          ) 
aquiz_camera_dat3        = rospy.Subscriber('/iris_triplecam_env/usb_cam_3/image_raw',  Image,          getVisual3          ) 
                                                                                                                            
#Publisher-Initialization+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++                    
cmd_cube_pub            = rospy.Publisher('/gazebo/set_model_state',                    ModelState,     queue_size=1        )
cmd_set_pos             = rospy.Publisher('/mavros/setpoint_position/local',            PoseStamped,    queue_size=1        )
rate = rospy.Rate(25)

#++++++++++++++++++++++++++++++Initialization Ends++++++++++++++++++++++++++++++++++
prevstamp = 0
# cmd_cube_pub.publish(PX4manipulation)
#ItStarts+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
while ~rospy.is_shutdown():
    sT6 = time.time() 
    if (clearanceCMD > 1900) & (firstdataget == False) :        # 레버 맨위에
        
        while (clearanceCMD > 1900):
            print 'pull down the trigger so it is on the second phase'
        pass

    elif (clearanceCMD > 1900) & (firstdataget == True): 
        print 'Cessation command detected, byebye'
        break

    curpose = [Lpose_x, Lpose_y, Lpose_z]
    # gather data of the frame and append'em into the thisrun data set
    

    if (savedflag == False) & (clearanceCMD > 1490):            # 레버 가운데
            
            #updated = gatherData_Whole( gatheredRun )
            #save_jason( updated )
            
            abcd = copy.copy(gatherData_thisframe( Stamp, \
                                        [csvSerializablize( MakeGS( VisualHexahedralize( view1 ) )), \
                                         csvSerializablize( MakeGS( VisualHexahedralize( view2 ) )), \
                                         csvSerializablize( MakeGS( VisualHexahedralize( view3 ) ))],\
                                         [ Lvel_x, Lvel_y, Lvel_z ] ) )
             
            gatherData_thisrun(abcd)
            
            savedflag = True
            if firstdataget == False : 
                firstdataget = True
                clearanceCMD = 1899
    print '++++++++++++++++++++++++++++++++',frameCounter
      
    if (clearanceCMD < 1490) :                                  # 레버 맨아래
        print 'saving'
        OneWitch()
        
        # cmd_cube_pub.publish(PX4manipulation)
        
        if initializeflag == False:
            sequenceCounter = sequenceCounter + 1
            
            save_csv('open')
            for framenum in range(frameCounter):
                save_csv(framenum, gatherData_thisrun('bringME',framenum))
                print 'saved' + '_' + '%d'%sequenceCounter + '_' + '%d'%(framenum+1) + '/' +'%d'%frameCounter
                # cmd_cube_pub.publish(PX4manipulation)
                if (clearanceCMD > 1900) & (firstdataget == True): 
                    print 'Cessation command detected, byebye'
                    break
            save_csv('close')
            gatherData_thisrun('clearCMD')
            initializeflag = True
        
        print 'initialization command is detected, switch the trigger to the middle'

    
    else:
        initializeflag = False
        print 'new sequence started'

    if( (isitLandedQuestionmark(curpose) == True) ):
        print 'landed properly' #는 local position이 순간이동하면 최신화가 안되기 때문에 뜨기 힘듬
    print 'CMD', clearanceCMD
    savedflag = False
    rate.sleep()
    print 'loop_dur : %f'%(time.time()-sT6)
    
quit()

